Retrieve job data from a PostgreSQL database and sort by date. 

In [3]:
import psycopg2
import pandas as pd
from urllib import parse
import os
import sys

# Add the path to the folder containing the env.py
sys.path.append('../')
from env import *

DATABASE_URL = os.environ['DATABASE_URL']
parse.uses_netloc.append('postgres')
url = parse.urlparse(DATABASE_URL)
DB_HOST = url.hostname
DB_PORT = url.port
DB_NAME = url.path[1:]
DB_USER = url.username
DB_PASSWORD = url.password

conn = psycopg2.connect(host=DB_HOST, port=DB_PORT,
                        database=DB_NAME, user=DB_USER, password=DB_PASSWORD)

cur = conn.cursor()

cur.execute(
    '''
        SELECT j.job_id, j.title, j.company, l.location_name as location, c.category_name as category, j.date, j.skills, j.link
        FROM jobs j
        JOIN locations l ON j.location_id = l.id
        JOIN job_categories c ON j.category_id = c.id
        '''
)
job_data = cur.fetchall()

# create pandas DataFrame
df = pd.DataFrame(job_data, columns=[
                  "job_id", "title", "company", "location", "category", "date", "skills", "link"])

cur.close()
conn.close()

df_sorted = df.sort_values('date', ascending=False)

df_sorted

,job_id,title,company,location,category,date,skills,link
293,3470051574,React Developer,Affinity,"Lisbon, Lisbon, Portugal",Serviços e consultoria de TI,2023-07-09,React,https://pt.linkedin.com/jobs/view/react-develo...
325,3659406705,JavaScript / TypeScript Engineer (React),Evolution,"Lisboa, Lisbon, Portugal",Serviços e consultoria de TI,2023-07-09,"React,Jest,Jenkins,Redux,Enzyme,iOS,Jira,JavaS...",https://pt.linkedin.com/jobs/view/javascript-t...
429,2871444461,Software Engineer - React Developer,GRiT Solutions,"Porto, Porto, Portugal",Serviços e consultoria de TI,2023-07-09,"React,Agile Methodology,CI/CD,JavaScript",https://pt.linkedin.com/jobs/view/software-eng...
381,3654252929,Frontend Software Engineer,HiBob,"Lisboa, Lisbon, Portugal",Desenvolvimento de software personalizado de s...,2023-07-09,"React,Angular,Microservices Architecture,CSS,T...",https://pt.linkedin.com/jobs/view/frontend-sof...
68,3442085243,Quant Engineer - Lisbon or Porto,BNP Paribas CIB,"Lisboa, Lisbon, Portugal",Serviços financeiros,2023-07-09,"C,Artificial Intelligence,DevOps,Go,JavaScript...",https://pt.linkedin.com/jobs/view/quant-engine...
...,...,...,...,...,...,...,...,...
318,3427792722,Oferta: Web Developer,KCS iT,"Porto, Porto, Portugal",Serviços e consultoria de TI,2022-12-11,"Angular,React,JavaScript",https://pt.linkedin.com/jobs/view/oferta-web-d...
216,3427793485,Oferta de emprego: Consultor Júnior (M/F) – Co...,Oficina de Competências,"Braga, Braga, Portugal",Serviços de recursos humanos,2022-12-11,,https://pt.linkedin.com/jobs/view/oferta-de-em...
479,3103152307,NodeJs Developer,Smart Consulting,"Porto, Portugal",Serviços e consultoria de TI,2022-07-26,"Scrum,JavaScript",https://pt.linkedin.com/jobs/view/nodejs-devel...
560,2650549046,UX/UI Designer,Smart Consulting,Lisbon Metropolitan Area,Serviços e consultoria de TI,2022-03-07,User Research,https://pt.linkedin.com/jobs/view/ux-ui-design...


## Data preprocessing

* Data profiling

In [4]:
import matplotlib.pyplot as plt
# View the first few rows of the DataFrame
df.head()

# Get summary statistics of the numeric columns
df.describe()

# Check the data types of each column
df.dtypes

# Check the number of missing values in each column
df.isnull().sum()


job_id      0
title       0
company     0
location    0
category    0
date        0
skills      0
link        0
dtype: int64

* Handle missing values

In [5]:
# Fill missing values with "N/A"
df_filled = df.fillna(value="N/A")

* Handle duplicates

In [6]:
# Drop duplicate rows
df.drop_duplicates()

# Count the number of duplicate rows
df.duplicated().sum()

0

* Prepare data for analysis (process date and location formats)

In [11]:
from geotext import GeoText


# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])


def extract_place_names(location_str):
    places = GeoText(location_str)
    city = places.cities[0] if places.cities else ''
    country = places.countries[0] if places.countries else ''
    return city, country


df[['city', 'country']] = df['location'].apply(
    lambda x: pd.Series(extract_place_names(x)))

         job_id                                 title  \
0    3645811837                          Data Analyst   
1    3654895534               Backend Engineer Python   
2    3633677699  Senior Software Engineer - Python SE   
3    3520146025              Django Backend Developer   
4    3194940087            Software Engineer (Python)   
..          ...                                   ...   
605  3632315061                    Junior SW Engineer   
606  3635280197         Senior Python Developer (m/f)   
607  3638490397                     Backend Developer   
608  3597476083                         Data Engineer   
609  3654261004                    Software Developer   

                                   company                   location  \
0                                 Celfocus   Lisbon, Lisbon, Portugal   
1                BytePitch - Software Labs     Porto, Porto, Portugal   
2                          ComplyAdvantage   Lisboa, Lisbon, Portugal   
3                      

In [13]:
df.head(20)

,job_id,title,company,location,category,date,skills,link,city,country
0,3645811837,Data Analyst,Celfocus,"Lisbon, Lisbon, Portugal",Serviços e consultoria de TI,2023-06-30,"SQL,PHP,Artificial Intelligence,Grafana,Tablea...",https://pt.linkedin.com/jobs/view/data-analyst...,Lisbon,Portugal
1,3654895534,Backend Engineer Python,BytePitch - Software Labs,"Porto, Porto, Portugal","Tecnologia, Informação e Internet",2023-06-09,"Scrum,DevOps,Scala,Fintech,AWS,Data Engineerin...",https://pt.linkedin.com/jobs/view/backend-engi...,Porto,Portugal
2,3633677699,Senior Software Engineer - Python SE,ComplyAdvantage,"Lisboa, Lisbon, Portugal",Desenvolvimento de software,2023-07-03,"Python,Kotlin,Natural Language Processing,Java",https://pt.linkedin.com/jobs/view/senior-softw...,Lisbon,Portugal
3,3520146025,Django Backend Developer,Probely,"Lisboa, Lisbon, Portugal",Segurança de redes e computadores,2023-03-09,"Cybersecurity,Security Testing,Django,Informat...",https://pt.linkedin.com/jobs/view/django-backe...,Lisbon,Portugal
4,3194940087,Software Engineer (Python),Revolut,Portugal,"Serviços e consultoria de TI, Desenvolvimento ...",2023-06-19,"Data Pipelines,Docker,Django,Data Privacy,Ansi...",https://pt.linkedin.com/jobs/view/software-eng...,,Portugal
5,3652435372,Senior Python Developer,Core Tech Recruitment,"Lisbon, Portugal",Recrutamento e seleção,2023-07-07,"Docker,Data Science,AWS,CI/CD,Python,Kubernete...",https://pt.linkedin.com/jobs/view/senior-pytho...,Lisbon,Portugal
6,3631040207,Python Developer,Noesis,"Lisboa, Lisbon, Portugal",Serviços de gestão estratégica,2023-06-08,"Python,Docker,CI/CD,Microservices",https://pt.linkedin.com/jobs/view/python-devel...,Lisbon,Portugal
7,3194935635,Software Engineer (Python),Revolut,"Porto, Porto, Portugal","Serviços e consultoria de TI, Desenvolvimento ...",2023-06-19,"Data Pipelines,Docker,Django,Data Privacy,Ansi...",https://pt.linkedin.com/jobs/view/software-eng...,Porto,Portugal
8,3628051380,team Python Developer,team.it,"Porto, Porto, Portugal",Serviços e consultoria de TI,2023-06-05,"Python,DevOps",https://pt.linkedin.com/jobs/view/team-python-...,Porto,Portugal
9,3493425989,Data Scientist (Andorra),Gauss & Neumann,"Lisbon, Lisbon, Portugal",N/A,2023-02-21,"PHP,Go,R,Python,SQL",https://pt.linkedin.com/jobs/view/data-scienti...,Lisbon,Portugal


Save preprocessed data

In [ ]:
import pickle


with open('processed_data.pkl', 'wb') as f:
    pickle.dump(df, f)